In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

# 포함할 abnormal return 컬럼
abn_return_cols = [
    "abn_ret_1m", "abn_ret_2m", "abn_ret_3m", "abn_ret_4m", "abn_ret_5m", "abn_ret_6m",
    "abn_ret_7m", "abn_ret_8m", "abn_ret_9m", "abn_ret_10m"
]

# 포함할 volume 컬럼
volume_cols = [
    "volume_minus_10m", "volume_minus_9m", "volume_minus_8m", "volume_minus_7m", "volume_minus_6m",
    "volume_minus_5m", "volume_minus_4m", "volume_minus_3m", "volume_minus_2m", "volume_minus_1m",
    "volume_1m", "volume_2m", "volume_3m", "volume_4m", "volume_5m", "volume_6m",
    "volume_7m", "volume_8m", "volume_9m", "volume_10m"
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"ar.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["telegram_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_1m,abn_ret_2m,abn_ret_3m,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,...,volume_3m,volume_4m,volume_5m,volume_6m,volume_7m,volume_8m,volume_9m,volume_10m,label,telegram_dummy
0,2021-01-04 10:08:00+00:00,0.75,-0.35,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
1,2021-01-04 10:08:00+00:00,-0.07,-0.32,-2.66,-2.49,-2.27,-1.24,-1.62,-1.37,-1.04,...,11510.0,2956.0,3751.0,1782.0,4513.0,337.0,34.0,1704.0,1,0
2,2021-01-04 10:08:00+00:00,0.75,-0.35,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
3,2021-01-04 10:14:00+00:00,0.10,-0.13,-2.60,-4.00,-3.64,-3.51,-3.64,-4.17,-4.70,...,167457.0,74079.0,15460.0,17484.0,11350.0,12145.0,38281.0,10831.0,1,0
4,2021-01-04 10:16:00+00:00,0.46,0.47,0.25,0.19,0.06,-0.24,-0.06,-0.24,-0.50,...,1588.0,1805.0,5776.0,6845.0,880.0,5369.0,3481.0,840.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,3.30,9.04,9.00,13.65,7.01,7.60,7.93,7.26,7.78,...,8934.0,19058.0,27278.0,14587.0,7554.0,6315.0,5149.0,2479.0,1,1
8652,2023-12-28 14:53:00+00:00,0.67,-0.64,0.24,0.29,0.21,0.21,0.23,0.27,0.27,...,82.0,2001.0,210.0,32.0,237.0,70.0,70.0,70.0,1,1
8653,2023-12-28 14:56:00+00:00,-0.01,0.42,0.42,0.44,0.48,0.48,0.45,0.45,0.42,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1
8654,2023-12-28 14:56:00+00:00,-0.01,0.42,0.42,0.44,0.48,0.48,0.45,0.45,0.42,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1


In [3]:
df_total = df_total.dropna()

In [4]:
for m in range(1, 11):
    minus_cols = [f"volume_minus_{x}m" for x in range(m, 0, -1)]
    df_total[f"cum_volume_minus_{m}m"] = df_total[minus_cols].sum(axis=1)

# 누적 거래량 (cum_volume_1m ~ cum_volume_10m)
for m in range(1, 11):
    plus_cols = [f"volume_{x}m" for x in range(1, m + 1)]
    df_total[f"cum_volume_{m}m"] = df_total[plus_cols].sum(axis=1)
    

# volume으로 시작하는 컬럼 모두 drop
volume_cols_to_drop = [col for col in df_total.columns if col.startswith("volume")]
df_total = df_total.drop(columns=volume_cols_to_drop)


/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_12717/512218477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total[f"cum_volume_minus_{m}m"] = df_total[minus_cols].sum(axis=1)
/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_12717/512218477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total[f"cum_volume_minus_{m}m"] = df_total[minus_cols].sum(axis=1)
/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_12717/512218477.py:3: SettingWithCopyWarning: 
A value is t

In [5]:
df_total.filter(regex="^cum_").head()

,cum_volume_minus_1m,cum_volume_minus_2m,cum_volume_minus_3m,cum_volume_minus_4m,cum_volume_minus_5m,cum_volume_minus_6m,cum_volume_minus_7m,cum_volume_minus_8m,cum_volume_minus_9m,cum_volume_minus_10m,cum_volume_1m,cum_volume_2m,cum_volume_3m,cum_volume_4m,cum_volume_5m,cum_volume_6m,cum_volume_7m,cum_volume_8m,cum_volume_9m,cum_volume_10m
0,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
2,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
3,549.0,557.0,658.0,811.0,1540.0,2617.0,3382.0,7552.0,7629.0,14141.0,172251.0,344502.0,511959.0,586038.0,601498.0,618982.0,630332.0,642477.0,680758.0,691589.0
4,18708.0,31931.0,38483.0,41085.0,45101.0,62675.0,71568.0,78630.0,83825.0,98085.0,6055.0,11175.0,12763.0,14568.0,20344.0,27189.0,28069.0,33438.0,36919.0,37759.0
5,2481.0,4602.0,5508.0,14969.0,19069.0,23070.0,25560.0,28946.0,31038.0,33680.0,2481.0,4962.0,7443.0,9924.0,12405.0,14886.0,17367.0,19848.0,22329.0,24810.0


In [6]:
import numpy as np

# delta_cum_volume_{m}m = (cum_volume_{m}m - cum_volume_minus_{m}m) / cum_volume_minus_{m}
for m in range(1, 11):
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

# min-max scaling for each delta_cum_volume_{m}m to [-1, 1]
for m in range(1, 11):
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
0,0.223019,-0.002257,-0.051197,-0.070336,-0.183510,-0.192928,-0.228230,-0.245325,-0.239524,-0.241210
2,0.223019,-0.002257,-0.051197,-0.070336,-0.183510,-0.192928,-0.228230,-0.245325,-0.239524,-0.241210
3,0.345005,0.380079,0.429372,0.415557,0.317633,0.262546,0.233811,0.154979,0.163281,0.098184
4,-0.352025,-0.362251,-0.333604,-0.337848,-0.387720,-0.390513,-0.400991,-0.390271,-0.383782,-0.403325
5,-0.237662,-0.250520,-0.195478,-0.275963,-0.349532,-0.349361,-0.344393,-0.341109,-0.333240,-0.336144


In [7]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
count,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000,8135.000000
mean,0.026726,-0.023686,-0.008444,-0.031050,-0.101705,-0.112920,-0.123726,-0.128946,-0.131929,-0.141225
std,0.292079,0.256599,0.237990,0.227819,0.230492,0.220443,0.212255,0.206735,0.201832,0.198535
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-0.198135,-0.214593,-0.183307,-0.197446,-0.267339,-0.270142,-0.274762,-0.274160,-0.273168,-0.279475
50%,0.001271,-0.053816,-0.037428,-0.062147,-0.136492,-0.145587,-0.155919,-0.160080,-0.162780,-0.172126
75%,0.226059,0.139326,0.139099,0.107874,0.036678,0.020136,0.002405,-0.007487,-0.013251,-0.026116
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
positive_df = df_total[df_total["label"] == 1]
positive_df["time_to_peak"] = positive_df[abn_return_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)

negative_df = df_total[df_total["label"] == -1]
negative_df["time_to_peak"] = negative_df[abn_return_cols].idxmin(axis=1).str.extract(r'(\d+)').astype(int)

/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_12717/2394544948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df["time_to_peak"] = positive_df[abn_return_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)
/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_12717/2394544948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df["time_to_peak"] = negative_df[abn_return_cols].idxmin(axis=1).str.extract(r'(\d+)').astype(int)


In [9]:
# df_total
# abn_ret만 써서 사용
# 10 min에서 하나라도 null 값 제외 (volume도 마찬가지)
# label: 1 > abn_ret argmax TTP 계산
# label: -1 > abn_ret argmin TTP 계산

#ln(TTP) = alpha + beta * telegram_dummy + gamma * min-max scale(ln(VolumeDiff)) + epsilon

# 회귀분석

### 📘 모형식

$$
\left| CAR_{post,i} - CAR_{pre,i} \right| = \alpha + \beta \cdot Telegram_i + \epsilon_i
$$

---

### 📊 변수 설명

**종속변수 (Dependent Variable)**  
이벤트 전후 CAR 변화폭의 절댓값:

$$
\left| CAR_{post,i} - CAR_{pre,i} \right|
$$


**설명변수 (Key Independent Variable)**  
텔레그램 도입 여부 변수:

$$
Telegram_i =
\begin{cases}
1, & \text{텔레그램 도입 이후 (After introduction)} \\
0, & \text{도입 이전 (Before introduction)}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{텔레그램 도입 이후와 이전 간 CAR 변화폭의 차이가 없다.}) \\
H_1 &: \beta < 0 \quad &(\text{텔레그램 도입 이후 CAR 변화폭이 감소했다.})
\end{aligned}
$$
---

### 📈 해석

이 회귀모형은 **텔레그램 도입 여부**가 이벤트 전후 누적초과수익률(CAR) 변화폭의 크기에 미치는 영향을 분석하기 위한 것입니다.  
β < 0 이고 통계적으로 유의하다면 귀무가설이 기각되어 공시 정보가 도입 이후 시장에 보다 신속하게 반영되어 정보 비효율성이 감소했음을 시사하며, 텔레그램 공시 알림 서비스가 시장 효율성 향상에 실질적으로 기여했음을 보여줍니다.


In [10]:
from hypothesis_test_TTP import regress_ln_ttp_with_volume, regress_ttp_with_volume, regress_ln_ttp, regress_ttp

summary = regress_ln_ttp_with_volume(positive_df)
summary = regress_ln_ttp_with_volume(negative_df)
df_total_ttp = pd.concat([positive_df, negative_df])
summary = regress_ln_ttp_with_volume(df_total_ttp)


--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const  1.0397*** 0.0186  55.790 0.000000
      telegram_dummy     0.0449 0.0251   1.786 0.074045
delta_cum_volume_10m -0.5323*** 0.0612  -8.702 0.000000
R-squared: 0.0163, Adj R-squared: 0.0158, n_obs: 4635

--- ln(time_to_peak) 회귀 (10m) ---
            variable      coef    std  t_stat  p_value
               const 1.4644*** 0.0562  26.070 0.000000
      telegram_dummy   -0.0234 0.0640  -0.366 0.714184
delta_cum_volume_10m   0.3928* 0.1863   2.108 0.035006
R-squared: 0.0069, Adj R-squared: 0.0040, n_obs: 700

--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const  1.0783*** 0.0177  61.038 0.000000
      telegram_dummy     0.0298 0.0235   1.268 0.204652
delta_cum_volume_10m -0.4947*** 0.0578  -8.563 0.000000
R-squared: 0.0136, Adj R-squared: 0.0133, n_obs: 5335


In [11]:
summary = regress_ttp_with_volume(positive_df)
summary = regress_ttp_with_volume(negative_df)
summary = regress_ttp_with_volume(df_total_ttp)


--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const  3.9584*** 0.0676  58.572 0.000000
      telegram_dummy    0.1971* 0.0932   2.114 0.034539
delta_cum_volume_10m -1.9684*** 0.2215  -8.885 0.000000
R-squared: 0.0164, Adj R-squared: 0.0160, n_obs: 4635

--- ln(time_to_peak) 회귀 (10m) ---
            variable      coef    std  t_stat  p_value
               const 5.4862*** 0.2229  24.608 0.000000
      telegram_dummy   -0.0003 0.2547  -0.001 0.998961
delta_cum_volume_10m    1.4427 0.7403   1.949 0.051332
R-squared: 0.0058, Adj R-squared: 0.0029, n_obs: 700

--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const  4.0960*** 0.0647  63.303 0.000000
      telegram_dummy     0.1488 0.0879   1.693 0.090531
delta_cum_volume_10m -1.8259*** 0.2111  -8.648 0.000000
R-squared: 0.0134, Adj R-squared: 0.0131, n_obs: 5335


In [12]:
summary = regress_ln_ttp(positive_df)
summary = regress_ln_ttp(negative_df)
summary = regress_ln_ttp(df_total_ttp)


--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 1.1091*** 0.0170  65.243 0.000000
telegram_dummy    0.0348 0.0253   1.376 0.168812
R-squared: 0.0004, Adj R-squared: 0.0002, n_obs: 4635

--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 1.3805*** 0.0385  35.841 0.000000
telegram_dummy   -0.0190 0.0639  -0.298 0.765636
R-squared: 0.0001, Adj R-squared: -0.0013, n_obs: 700

--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 1.1489*** 0.0157  73.342 0.000000
telegram_dummy    0.0194 0.0236   0.824 0.409983
R-squared: 0.0001, Adj R-squared: -0.0001, n_obs: 5335


In [13]:
summary = regress_ttp(positive_df)
summary = regress_ttp(negative_df)
summary = regress_ttp(df_total_ttp)


--- time_to_peak 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 4.2151*** 0.0626  67.306 0.000000
telegram_dummy    0.1599 0.0939   1.703 0.088478
R-squared: 0.0006, Adj R-squared: 0.0004, n_obs: 4635

--- time_to_peak 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 5.1782*** 0.1534  33.748 0.000000
telegram_dummy    0.0158 0.2540   0.062 0.950442
R-squared: 0.0000, Adj R-squared: -0.0014, n_obs: 700

--- time_to_peak 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 4.3563*** 0.0583  74.707 0.000000
telegram_dummy    0.1106 0.0883   1.253 0.210233
R-squared: 0.0003, Adj R-squared: 0.0001, n_obs: 5335
